In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
from requests_html import HTMLSession, AsyncHTMLSession

In [3]:
rejected = [
    'wa.link',
    'mailto:',
    't.me',
    'tally.so',
    'discord.gg',
    'arxiv.org',
    'javascript:'
]

In [4]:
async def get_html(url):
    session = AsyncHTMLSession(verify = False)
    result = None
    try:
        
        r = await session.get(url, verify = False, timeout = 10)
        await r.html.arender(timeout = 10)
        
        result = r.html.html
    except Exception as e:
        pass
    
    await session.close()
    return result


def get_links(html):
    soup = BeautifulSoup(html, 'html.parser')
    links = []
    for link in soup.find_all('a'):
        links.append(link.get('href'))
    return links

In [6]:
import asyncio

visited_pages = set(['https://jwp.gov.my/'])

async def crawl(base_url):
    if base_url.endswith('.pdf'):
        return
    
    html = await get_html(base_url)
    
    if html is None:
        return

    visited_pages.add(base_url)
    links = get_links(html)
    for link in links:
        try:
            if any([r in link for r in rejected]):
                continue

            if not link.startswith('http'):
                link = urljoin(base_url, link)

            link = link.split('#')[0]
            link = link.split('?')[0]

            if link not in visited_pages:
                visited_pages.add(link)
        except Exception as e:
            pass

In [7]:
from tqdm import tqdm

batch_size = 10

for no in range(3):
    l = list(visited_pages)
    print(no, len(l))
    for i in tqdm(range(0, len(l), batch_size)):
        b = l[i: i + batch_size]
        await asyncio.gather(*[crawl(url) for url in b])

0 1


100%|█████████████████████████████████████████████| 1/1 [00:08<00:00,  8.64s/it]


1 91


 20%|████████▊                                   | 2/10 [00:17<01:11,  8.95s/it]Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error Target.detachFromTarget: Target closed.')>
pyppeteer.errors.NetworkError: Protocol error Target.detachFromTarget: Target closed.
Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id
 30%|█████████████▏                              | 3/10 [00:29<01:10, 10.06s/it]Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error Target.detachFromTarget: Target closed.')>
pyppeteer.errors.NetworkError: Protocol error Target.detachFromTarget: Target closed.
Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessage

2 2786


  7%|██▊                                       | 19/279 [03:34<50:23, 11.63s/it]Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id
 12%|█████                                     | 34/279 [06:40<54:06, 13.25s/it]Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id
 14%|█████▋                                    | 38/279 [07:21<43:22, 10.80s/it]Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No se

 80%|████████████████████████████████▌        | 222/279 [47:08<12:59, 13.67s/it]Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id
Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id
 81%|█████████████████████████████████        | 225/279 [47:48<11:29, 12.77s/it]Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id
 90%|████████████████████████████████████▋    | 250/279 [52:

In [9]:
import json

with open('url-jwp.gov.my.json', 'w') as fopen:
    json.dump(list(visited_pages), fopen)